In [ ]:
import os
import random
import itertools
from PIL import Image
from tqdm import tqdm
from itertools import combinations
import pandas as pd
import numpy as np
from fpt.data import join_face_df
from fpt.split import read_split
from fpt.path import DTFR, DATA
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from matplotlib import rc

font_path = "/usr/share/fonts/NanumFont/NanumGothicBold.ttf"
fontprop = fm.FontProperties(fname=font_path, size=18)

# Set global figure background color
# plt.rcParams["figure.facecolor"] = "white"
plt.style.use('seaborn-whitegrid')  # 스타일 적용

# sns.set(style="white")

# 한글 폰트 설정
font_name = fm.FontProperties(
    fname="/usr/share/fonts/NanumFont/NanumGothicBold.ttf"
).get_name()
rc("font", family=font_name)
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
def add_extension(path):
    if os.path.exists(path + ".jpg"):
        return path + ".jpg"
    elif os.path.exists(path + ".png"):
        return path + ".png"
    else:
        raise RuntimeError('No file "%s" with extension png or jpg.' % path)

In [ ]:
# 이미지와 정보를 보여주는 함수 정의
def show_image_and_info(
    df,
    pair_name,
):
    fig = plt.figure(figsize=(10, 4))
    fig.suptitle(pair_name.upper())  # 전체 figure에 대한 제목 설정
    for i in range(10):
        for j in range(2):
            ax = fig.add_subplot(2, 10, j * 10 + i + 1)  # (2-j) * 10 + i + 1
            # 이미지를 읽어옴
            img = Image.open(df.iloc[i][["l", "r"][j] + "image"])
            ax.imshow(np.array(img))
            # ax.axis("off")  # 축 제거
            ax.set_title(
                "\n".join(
                    df.iloc[i][
                        [
                            "age_group_" + ["left", "right"][j],
                            "gender_" + ["left", "right"][j],
                            "target_" + ["left", "right"][j],
                        ]
                    ].values
                )
            )
            if j == 1:
                if pair_name.lower().startswith("wrong"):
                    ax.set_xlabel(
                        "False Neg." if df.iloc[i]["issame"] else "False Pos.",
                        color="green" if df.iloc[i]["issame"] else "red",
                        fontsize=10,
                    )
                else:
                    ax.set_xlabel(
                        "Match" if df.iloc[i]["issame"] else "Mismatch",
                        color="green" if df.iloc[i]["issame"] else "red",
                        fontsize=10,
                    )

            # 축 제거
            ax.set_xticks([])  # x 축의 틱(tick) 제거
            ax.set_yticks([])  # y 축의 틱(tick) 제거
            for spine in ax.spines.values():  # 모든 축의 선 제거
                spine.set_visible(False)

    plt.tight_layout()
    image_path = f"output/pairs/{pair_name}"
    plt.savefig(image_path, facecolor="w")
    plt.show()

In [ ]:
face = join_face_df(DTFR, "aihub_family")
test_uuids = read_split("test")
x_test = face.loc[test_uuids]
x_test = x_test[x_test.age_group != "above"]
x_test = x_test.reset_index().reset_index().set_index("uuid")
aihub_dir = DATA / "face-image/test_aihub_family"

In [ ]:
pair_name = "basic-fc"
pairs_path = f"/home/jongphago/family-photo-tree/data/pairs/test/pairs_{pair_name.upper()}.txt"

In [ ]:
def get_distance_list(distance_path):
    with open(distance_path, "r") as f:
        l = f.readline().rstrip()
        best_distance = float(l.split(", ")[1])
        lines = f.readlines()
        distances = [float(l.rstrip()) for l in lines]
        assert len(distances) == 6000
    return distances, best_distance


def get_path(pair_name, model_type, checkpoint):
    distance_path = DATA / f"distance/{model_type}/{checkpoint}/{pair_name}.txt"
    pairs_path = f"pairs/test/pairs_{pair_name}.txt"
    return distance_path, pairs_path

In [ ]:
pair_names = [
    "BASIC-G",
    "BASIC-GN",
    "BASIC-GC",
    "BASIC-A",
    "BASIC-AN",
    "BASIC-AC",
    "BASIC-F",
    "BASIC-FN",
    "BASIC-FN",
    "BASIC-FC",
    "FAMILY-A",
    "FAMILY-CA",
    "FAMILY-G",
    "FAMILY-CG",
    "FAMILY-AG",
    "FAMILY-CAG",
    "PERSONAL-A",
    "PERSONAL-AC",
    "Age",
    "Family",
    "Individuals",
]

# Loop

In [ ]:

model_type, checkpoint = "single-fr-ver-1", "230529_0140"
is_wrong = True

for pair_name in pair_names:
    # get path
    distance_path, pairs_path = get_path(pair_name, model_type, checkpoint)

    # Distance array
    distances, best_distance = get_distance_list(distance_path)
    distances = np.array([distances, np.array(distances) > best_distance], dtype=int).T


    # for pair_name in pair_names:
    pairs_path = (
        f"/home/jongphago/family-photo-tree/data/pairs/test/pairs_{pair_name.upper()}.txt"
    )
    pairs = []
    is_sames = []
    path_list = []
    issame_list = []
    with open(pairs_path, "r") as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            is_sames.append(True if len(pair) == 3 else False)
            if len(pair) == 3:
                target = pair[0]
                pair.insert(2, target)
            pairs.append(pair)
    for pair in pairs:
        if len(pair) == 3:
            path0 = add_extension(os.path.join(aihub_dir, pair[0], pair[1]))
            path1 = add_extension(os.path.join(aihub_dir, pair[0], pair[2]))
            issame = True
        elif len(pair) == 4:
            path0 = add_extension(os.path.join(aihub_dir, pair[0], pair[1]))
            path1 = add_extension(os.path.join(aihub_dir, pair[2], pair[3]))
            issame = False
        if os.path.exists(path0) and os.path.exists(
            path1
        ):  # Only add the pair if both paths exist
            path_list.append((path0, path1))
            issame_list.append(issame)
            
    pairs = np.array(pairs, dtype=object)
    is_sames = np.array(is_sames, dtype=np.int64)[:, np.newaxis]
    columns = ["issame", "ltarget", "luuid", "rtarget", "ruuid", "distance", "is_larger"]
    pairs_df = pd.DataFrame(np.hstack((is_sames, pairs, distances)), columns=columns)
    image_path_df = pd.DataFrame(path_list, columns=["limage", "rimage"])
    pairs_df = pairs_df.merge(image_path_df, left_index=True, right_index=True, how="outer")

    # df
    temp_x_test = x_test.reset_index().set_index("index")  # (8147, 15)
    temp_merged = pd.merge(
        pairs_df,
        temp_x_test,
        left_on="luuid",
        right_on="uuid",
    )
    df = pd.merge(
        temp_merged,
        temp_x_test,
        left_on="ruuid",
        right_on="uuid",
        suffixes=["_left", "_right"],
    )
    
    if is_wrong:
        wrong = df[df.issame + df.is_larger != 1]
        df_sample = wrong.sample(10)
        pair_name = f"wrong/{model_type.split('-')[0]}/{pair_name}"
    else:
        # 데이터프레임에서 랜덤하게 10개의 행을 선택
        df_sample = df.sample(10)
    df_sample.sort_values('issame', inplace=True)

    # 함수를 사용하여 이미지와 정보를 보여줌
    show_image_and_info(df_sample, pair_name)